In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import neo
from scipy.signal import medfilt
from scipy.ndimage import label
from sklearn.cluster import KMeans

In [2]:
# read in the data
data_file = 'example.abf'
r = neo.AxonIO(filename=data_file)
block = r.read_block()
segment = block.segments[0]

nerve,muscle = segment.analogsignals
nerve = np.asarray(nerve[10000:100000])

In [3]:
# adjust for drift in baseline of signal; this may take a bit of time
baseline = medfilt(nerve, kernel_size=801)
nerve -= baseline

In [4]:
# compute the values in the signal that pass some threshold
direction = 1 # 1 for upward, -1 for downward
n_std = 2.5
thresh = nerve.mean() + nerve.std() * n_std
above_thresh = direction * nerve > thresh

In [5]:
# split the above_thresh signal into connected segments
labs,nlabs = label(above_thresh)

peak_idxs = [] # stores the indices of peaks

# iterate through each segment
for l in range(1,nlabs+1):
    
    # find the indices of this segment
    idxs = np.where(labs==l)[0]
    
    # extract the signal values at these idxs
    vals = nerve[idxs]
    
    # select the index corresponding to the peak signal value
    peak_idx = idxs[np.argmax(vals)]
    
    peak_idxs.append(peak_idx)

peak_heights = nerve[peak_idxs]

In [6]:
# display the discovered peaks
pl.plot(nerve)
pl.plot(peak_idxs, peak_heights, 'go')

In [7]:
# display the distribution of peak values (might help you see clusters, and thus select k for k-means)
pl.hist(peak_heights, bins=100)
pl.xlabel('AP magnitude')
pl.ylabel('Count')

In [8]:
n_clusters = 5

# reshape the data to the shape (n_samples, n_features) -- required for scikit-learn
X = peak_heights.reshape([-1,1])
# run k-means clustering
km = KMeans(n_clusters=n_clusters).fit(X)

In [9]:
# display the nerve and the peaks coloured by cluster
pl.plot(nerve, color='gray')
pl.scatter(peak_idxs, peak_heights, c=km.labels_)